<a href="https://colab.research.google.com/github/anaguilarar/agwise_data_sourcing/blob/main/GEEElevation_data_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ag-Wise Data Sourcing

## Digital Elevation Model Downloader

This tutorial explains how to use this notebook to download the digital elevation model (DEM) data from google earth engine

In [ ]:
import os

if not os.path.exists('/content/agwise_data_sourcing'):
  !git clone https://github.com/anaguilarar/agwise_data_sourcing.git
  os.chdir('/content/agwise_data_sourcing')
else:
  os.chdir('/content/agwise_data_sourcing')

## Workflow Overview
1. **Country Example Configuration** – Select area and product to download.
2. **DEM Data visualization** – Visualize the digital elevation model.

### Country Example Configuration


In this section, you will set the administration boundary for your analysis. Modify the dictionary below to match your region and product of interest.


- `ADM0_NAME` define the administrative levels.


In [ ]:
### INITIAL configuration

configuration = {
    'GENERAL_SETTINGS':{
      'ee_project_name': 'ee-anaguilarar',
      'output_path': 'runs'
      },
    'DATA_DOWNLOAD':
    {
      'ADM0_NAME': 'Kenya',
      'ADM1_NAME': 'Kericho',
      'ADM2_NAME': None,
      'product': 'dem',
      'source': 'nasadem'

    },
    'OUTPUT':
      {
        'path': 'dem',
        'resolution': 250
      }
}


In [ ]:
from gee_datasets.dem import GEEdem
import ee
import geemap

ee.Initialize(project=configuration['GENERAL_SETTINGS']['ee_project_name'])


### Data Downloading


This section connects to Google Earth Engine, defines your region of interest, and retrieves the terrain values.


Steps:
1. **Initialize Google Earth Engine (GEE)** with your project.
2. **Create the downloader object** (`GEEdem`). Define the sorce product used for downloading.
3. **Run the query** to retrieve the imagery.

In [ ]:
data_downloader = GEEdem(configuration['DATA_DOWNLOAD']['ADM0_NAME'])

data_downloader.list_of_products

In [ ]:
data_downloader.initialize_query(configuration['DATA_DOWNLOAD']['source'])
band_names = data_downloader.query.bandNames().getInfo()
band_names

In [ ]:

# Create a map
Map = geemap.Map(center=[-1.37, 38.01], zoom=6)

# Define visualization parameters

vis_parameters = {'min': 0, 'max': 3000,
 'palette': ['5d5851','635a4b','6a5b44','715c3d','785e36','7e5f30','856129','8c6222','92641c','996515','a0660e','a66808','ad6901']}
# Add the image layer
Map.addLayer(data_downloader.query.select(band_names[0]), vis_parameters, band_names[0])
# Display the map
Map

### Download data for a specific administrative level

You can target data at different administrative levels using the configuration keys:

1. Set `ADM0_NAME` for the country (required).
2. Set `ADM1_NAME` for the first-level admin (province/state) if you want a subregion.
3. Set `ADM2_NAME` for the district/municipality if available and needed.


**Example configuration (Kenya, Coast province):**


```python
configuration['DATA_DOWNLOAD'].update({
'ADM0_NAME': 'Kenya',
'ADM1_NAME': 'Kericho',
'ADM2_NAME': None,

})

In [ ]:

dem_image = data_downloader.get_adm_level_data(adm_level='ADM1', feature_name = configuration['DATA_DOWNLOAD']['ADM1_NAME'])

Map = geemap.Map(center=[-0.5, 35.51], zoom=9)
Map.addLayer(dem_image.select(band_names[0]), vis_parameters, band_names[0])

# Display the map
Map

In [ ]:
### Download to local store
import os

## Reproject to epsg 4326

output_fn = os.path.join(configuration['OUTPUT']['path'],  'dem.tif')
if not os.path.exists(configuration['OUTPUT']['path']): os.mkdir(configuration['OUTPUT']['path'])
scale = configuration['OUTPUT']['resolution'] = 250 ## there are problem with a bigger resolution
data_downloader.download_data(dem_image, output_fn,  scale = configuration['OUTPUT']['resolution'])
band_names = dem_image.bandNames().getInfo()
band_names


In [ ]:
!pip install rioxarray

In [ ]:
### export as dataframe
import xarray
import rioxarray as rio

raster_list = [os.path.join('dem',i) for i in os.listdir('dem') if i.endswith('tif')]

xrdata = rio.open_rasterio(raster_list[0])
xrdata['band'] =  band_names

xrdata.name = 'value'
xrdata.to_dataframe().reset_index().drop(columns='spatial_ref')

### Extracting data using coordinate

In [ ]:
coordinate = [37.8, -1.4] # long and lat
data_downloader = GEEdem(configuration['DATA_DOWNLOAD']['ADM0_NAME'])
scale = configuration['OUTPUT']['resolution'] = 30
df = data_downloader.terraindata_using_point(coordinate,  scale = configuration['OUTPUT']['resolution'])

df